In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from import_casa import casa, cano

In [3]:
TARGET_DIR = "20210406"
result_dir = casa.get_data_path() / f"annot_data/annotated_data_bkup/{TARGET_DIR}"
json_paths = [x for x in result_dir.iterdir() if x.suffix==".json"]

In [4]:
with json_paths[1].open("r", encoding="UTF-8") as fin:
    annots = json.load(fin)

In [5]:
list(annots[0].keys())

['completions', 'data', 'id']

In [6]:
len(annots)

382

In [7]:
aspect_list = []

In [8]:
for annot_i, annot_x in tqdm(enumerate(annots)):
    ctxs = [x for x in annot_x["completions"][0]["result"]  if x.get("from_name", "").startswith("ctx-sentiment")]
    if ctxs:
        print(annot_i)
        aspects = cano.process_thread_annotations(annot_x)
        break
        # aspect_list.extend(aspects)

1it [00:00, ?it/s]

1


In [9]:
aspect_list = []
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        annots = json.load(fin)
    for annot_i, annot_x in enumerate(annots):        
        aspects = cano.process_thread_annotations(annot_x)        
        aspect_list.extend(aspects)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 48.64it/s]

安平區
台北市區
499之亂後
NP
日本國際漫遊
成功嶺
海景


In [10]:
len(aspect_list)

1651

In [20]:
import pandas as pd
data_items = []
for aspect_x in aspect_list:
    batch_idx = aspect_x.batch_idx
    thread_idx = aspect_x.thread_idx
    serial = aspect_x.serial
    aspect_tuple = aspect_x.make_tuple()
    memo = aspect_x.memo
    ent_rawtext = aspect_x.raw_text(cano.AspectEnum.Entity)
    attr_rawtext = aspect_x.raw_text(cano.AspectEnum.Attribute)
    is_context = aspect_x.has_context_only
    if all(not x.strip() for x in aspect_tuple[0:3]):
        continue
    data_items.append((batch_idx, serial, thread_idx, is_context,
                      *aspect_tuple, ent_rawtext, attr_rawtext))
    

In [21]:
aspect_df = pd.DataFrame(data_items, 
             columns=["batch_idx", "serial", "thread_idx", "is_context", "ent_norm", "attr_norm", "evaltext", 
                      "rating", "ent_rawtext", "attr_rawtext"])
aspect_df.to_csv(result_dir/f"aspect_tuples_{TARGET_DIR}.csv", encoding="UTF-8", index=False)

In [22]:
aspect_df

,batch_idx,serial,thread_idx,is_context,ent_norm,attr_norm,evaltext,rating,ent_rawtext,attr_rawtext
0,0,43,3260,True,臺灣之星,[通訊]頻段,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,1,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
1,0,7,281,False,中華電信,[通訊]國內電信漫遊,很可以,4,中華,訊號
2,0,13,932,False,臺灣之星,[通訊]網速,不限速卡上傳可,3,台星,21M
3,0,1,114,True,中華電信,,中華、遠傳，蓋一座拆一座,1,中華、遠傳，蓋一座拆一座,中華、遠傳，蓋一座拆一座
4,0,10,674,False,臺灣大哥大,[通訊]國內電信漫遊,越來越爛,1,台灣大哥大,收訊
...,...,...,...,...,...,...,...,...,...,...
1582,6,440,23817,False,,[資費]月租費,,-1,,5G非吃到飽
1583,6,440,23817,True,中華電信,[資費]月租費,中華+台哥大合體 & 在5G非吃到飽部分比較有創意,4,中華+台哥大合體 & 在5G非吃到飽部分比較有創意,中華+台哥大合體 & 在5G非吃到飽部分比較有創意
1584,6,36,2460,False,中華電信,[通訊]網速,很奇怪,2,中華電信,網路
1585,6,99,6196,False,中華電信,[通訊]涵蓋,比較好,4,中華,收訊


In [16]:
[x for x in range(100) if annots[x]["data"]["thread_idx"] == 16514]

[9]

In [20]:
aa, dbg = cano.process_thread_annotations(annots[9], True)  

In [24]:
len(aspect_list)

521

In [25]:
aa[-1].make_tuple()

('', '', '', -1)